<a href="https://colab.research.google.com/github/davidlealo/vocacional-test/blob/main/mentoria_vocacional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explicación del Código de Mentoria Vocacional

Este documento describe un cuaderno Jupyter (`mentoria_vocacional.ipynb`) diseñado para implementar un sistema de mentoría pedagógica utilizando Azure Cognitive Search y Azure OpenAI. A continuación, se detalla cada sección del código, explicando su propósito y funcionalidad.

## 1. Instalación de Dependencias

```python
!pip install python-dotenv requests openai --upgrade
```

- **Propósito**: Instala las bibliotecas necesarias para ejecutar el código.
- **Detalles**:
  - `python-dotenv`: Permite cargar variables de entorno desde un archivo `.env`.
  - `requests`: Facilita realizar solicitudes HTTP a la API de Azure Search.
  - `openai`: Proporciona acceso a la API de Azure OpenAI para interactuar con modelos de lenguaje.
  - El flag `--upgrade` asegura que se instalen las versiones más recientes de estas bibliotecas.

**Salida**:
Muestra la instalación exitosa de las dependencias y sus versiones, junto con las subdependencias requeridas (como `charset_normalizer`, `idna`, etc.).

## 2. Carga de Variables de Entorno

```python
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests

load_dotenv()

AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]
```

- **Propósito**: Configura el entorno cargando credenciales y configuraciones desde un archivo `.env`.
- **Detalles**:
  - `load_dotenv()` lee las variables de entorno desde el archivo `.env`.
  - Se asignan variables para:
    - **Azure Cognitive Search**: Clave API (`AZURE_SEARCH_API_KEY`), endpoint (`AZURE_SEARCH_ENDPOINT`) y nombre del índice (`AZURE_SEARCH_INDEX_NAME`).
    - **Azure OpenAI**: Clave API (`AZURE_OPENAI_API_KEY`), endpoint (`AZURE_OPENAI_ENDPOINT`) y nombre del despliegue del modelo (`AZURE_OPENAI_DEPLOYMENT_NAME`).
  - Estas variables son esenciales para autenticar y conectar con los servicios de Azure.

## 3. Función para Buscar Documentos en Azure Search

```python
def search_documents(query, top_k=5):
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }
    payload = {
        "search": query,
        "top": top_k
    }
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()
    return [doc.get("content", "") for doc in results.get("value", [])]
```

- **Propósito**: Realiza una búsqueda de documentos en Azure Cognitive Search basada en una consulta de texto.
- **Detalles**:
  - **Parámetros**:
    - `query`: Texto de la consulta de búsqueda.
    - `top_k`: Número máximo de documentos a recuperar (por defecto, 5).
  - Construye una URL para la API de Azure Search utilizando el endpoint y el nombre del índice.
  - Configura los encabezados HTTP con el tipo de contenido (`application/json`) y la clave API.
  - Envía una solicitud POST con un cuerpo JSON que incluye la consulta (`search`) y el límite de resultados (`top`).
  - Maneja errores con `raise_for_status()` y devuelve una lista con el contenido (`content`) de los documentos recuperados.
  - Si el campo `content` no existe en un documento, se devuelve una cadena vacía.

**Variante con Depuración**:
En una celda posterior, se modifica esta función para incluir depuración:

```python
def search_documents(query, top_k=10):
    ...
    for doc in results.get("value", []):
        print("Campos disponibles:", doc.keys())
        break
    return [doc.get("content", "") for doc in results.get("value", [])]
```

- **Cambios**:
  - Aumenta `top_k` a 10 por defecto.
  - Imprime las claves de los documentos recuperados para inspeccionar los campos disponibles (como `@search.score`, `chunk_id`, `parent_id`, `chunk`, `title`, `text_vector`).
  - Opcionalmente, permite usar búsqueda semántica (comentada) con `"queryType": "semantic"`.

## 4. Configuración del Cliente de Azure OpenAI

```python
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)
```

- **Propósito**: Inicializa el cliente para interactuar con el modelo de Azure OpenAI (Phi-4).
- **Detalles**:
  - Usa la clave API, la versión de la API (`2023-08-01-preview`) y el endpoint configurados previamente.
  - Este cliente se utiliza para generar respuestas basadas en prompts.

## 5. Función para Generar Respuestas con Contexto

```python
def generate_answer(question, context):
    system_prompt = "Eres un mentor pedagógico. Responde de forma clara, breve y útil basado en el contexto."
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"
    
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=500
    )
    return response.choices[0].message.content
```

- **Propósito**: Genera una respuesta utilizando el modelo de Azure OpenAI, combinando una pregunta y un contexto recuperado.
- **Detalles**:
  - **Parámetros**:
    - `question`: La pregunta del usuario.
    - `context`: Texto recuperado de Azure Search.
  - Define un `system_prompt` que instruye al modelo a actuar como un mentor pedagógico, proporcionando respuestas claras y útiles.
  - Construye un `user_prompt` que incluye la pregunta y el contexto.
  - Llama a la API de chat completions con:
    - El nombre del despliegue del modelo.
    - Una lista de mensajes (system y user).
    - Configuraciones: `temperature=0.3` (para respuestas más deterministas) y `max_tokens=500` (límite de tokens en la respuesta).
  - Devuelve el contenido de la primera opción de la respuesta generada.

## 6. Ejecución de Ejemplos

El código incluye varias celdas que prueban el sistema con diferentes preguntas. A continuación, se describen los casos de uso:

### Ejemplo 1: Pregunta sobre Valentina y el Financiamiento de la Educación

```python
pregunta = "¿Qué dijo Valentina sobre el financiamiento de la educación?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)
print("Respuesta del asistente:\n", respuesta)
```

- **Propósito**: Busca documentos relacionados con la pregunta y genera una respuesta.
- **Salida**:
  - Los documentos recuperados están vacíos, por lo que la respuesta indica que falta contexto:  
    _"Parece que no has proporcionado el contexto específico en el que Valentina habló sobre el financiamiento de la educación..."_

### Ejemplo 2: Pregunta sobre Desafíos de Estudiantes Rurales según David

```python
pregunta = "¿Qué desafíos enfrentan los estudiantes en zonas rurales según David?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)
print("Respuesta del asistente:\n", respuesta)
```

- **Salida**: Similar al ejemplo anterior, no se encuentra contexto relevante, y la respuesta solicita más información:  
  _"Lamento, pero no puedo proporcionar una respuesta directa a tu pregunta ya que no has proporcionado el contexto o el texto en el que se menciona a David..."_

### Ejemplo 3: Pregunta sobre `proyectate.info`

```python
pregunta = "¿Qué es proyectate.info?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)
print("Respuesta del asistente:\n", respuesta)
```

- **Salida**: No se encuentra información sobre `proyectate.info`, y la respuesta refleja esto:  
  _"Parece que te refieres a 'projectate.info', pero no tengo información específica sobre un sitio web o recurso con ese nombre..."_

### Ejemplo 4: Pregunta sobre Pérdida de Beneficios del Crédito Universitario

```python
pregunta = "¿Qué debo hacer si perdí mis beneficios del crédito universitario?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
print("\n--- Documentos recuperados ---")
print(contexto[:1000])
respuesta = generate_answer(pregunta, contexto)
print("\n--- Respuesta del asistente ---")
print(respuesta)
```

- **Salida**:
  - Los documentos recuperados están vacíos, pero el modelo genera una respuesta general basada en conocimiento implícito, sugiriendo pasos como contactar al Centro de Ayuda Estudiantil, revisar informes de crédito, y buscar asesoría legal.
  - Esto indica que el modelo puede proporcionar respuestas útiles incluso sin contexto específico, aunque no se basa en documentos recuperados.

### Ejemplo 5: Múltiples Preguntas en un Bucle

```python
preguntas = [
    "¿Qué es Proyectate?",
    "¿Qué dijo Valentina sobre el crédito universitario?",
    "¿Qué recomendación dio David a los estudiantes rurales?"
]
for pregunta in preguntas:
    print(f"\n🔹 Pregunta: {pregunta}")
    documentos = search_documents(pregunta)
    contexto = "\n\n".join(documentos)
    respuesta = generate_answer(pregunta, contexto)
    print("Respuesta:", respuesta)
```

- **Propósito**: Procesa múltiples preguntas en un bucle, mostrando los resultados de cada una.
- **Salida**:
  - Todas las respuestas indican falta de contexto, solicitando más información para proporcionar respuestas precisas.
  - Esto sugiere que el índice de Azure Search no contiene documentos relevantes para estas preguntas.

### Ejemplo 6: Preguntas sobre Personas y el Chat de IA

```python
preguntas = [
    "Quién es Valentina Gran de Fundación Por Una Carrera?",
    "Quién es David Leal de Innovacien?",
    "¿Qué es el chat de Inteligencia artificial que se presentó en la conversación?"
]
for pregunta in preguntas:
    print(f"\n🔹 Pregunta: {pregunta}")
    documentos = search_documents(pregunta)
    contexto = "\n\n".join(documentos)
    respuesta = generate_answer(pregunta, contexto)
    print("Respuesta:", respuesta)
```

- **Salida**:
  - **Valentina Gran**: Se describe como una defensora de la igualdad de género y fundadora de Fundación Por Una Carrera, con detalles sobre su trabajo desde 2006.
  - **David Leal**: Identificado como CEO de Innovacien, una startup de IA, con énfasis en su liderazgo en transformación digital.
  - **Chat de IA**: No se encuentra contexto, y la respuesta solicita más detalles sobre la conversación referida.
  - **Nota**: Las respuestas sobre Valentina y David parecen basarse en conocimiento del modelo, no en documentos recuperados, ya que el contexto está vacío.

## 7. Funciones para Búsqueda con Vectores

### Generación de Embeddings

```python
def get_query_embedding(text):
    response = client.embeddings.create(
        model=os.environ["AZURE_OPENAI_EMBEDDING"],
        input=text
    )
    return response.data[0].embedding
```

- **Propósito**: Genera un embedding (vector numérico) para un texto dado usando un modelo de embeddings de Azure OpenAI.
- **Detalles**:
  - Usa la variable de entorno `AZURE_OPENAI_EMBEDDING` para especificar el modelo de embeddings.
  - Toma un texto (`text`) como entrada y devuelve su representación vectorial.

### Búsqueda con Vectores

```python
def search_documents_with_vectors(query, top_k=3):
    embedding = get_query_embedding(query)
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }
    payload = {
        "vectorQueries": [
            {
                "vector": embedding,
                "k": top_k,
                "fields": "content"
            }
        ],
        "select": "content,source,@search.score"
    }
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()
    return [doc["content"] for doc in results.get("value", [])]
```

- **Propósito**: Realiza una búsqueda basada en vectores en Azure Search, utilizando el embedding de la consulta.
- **Detalles**:
  - Genera el embedding de la consulta con `get_query_embedding`.
  - Construye una solicitud a Azure Search con un `vectorQueries` que incluye:
    - El vector de la consulta (`vector`).
    - El número de resultados a recuperar (`k`, por defecto 3).
    - El campo a buscar (`fields: content`).
  - Especifica los campos a devolver (`select: content,source,@search.score`).
  - Devuelve una lista con el contenido de los documentos recuperados.
- **Nota**: Esta función no se utiliza en los ejemplos proporcionados, pero está diseñada para búsquedas semánticas más avanzadas.

## Observaciones Generales

- **Estructura**: El cuaderno está organizado en celdas que progresivamente configuran el entorno, definen funciones y ejecutan ejemplos.
- **Dependencias de Azure**: El código depende de servicios de Azure (Cognitive Search y OpenAI), requiriendo credenciales válidas y un índice poblado.
- **Limitaciones Observadas**:
  - En la mayoría de los ejemplos, los documentos recuperados están vacíos, lo que sugiere que el índice de Azure Search no contiene datos relevantes o que las consultas no coinciden con el contenido indexado.
  - Las respuestas generadas a menudo se basan en el conocimiento del modelo en lugar de los documentos recuperados.
- **Depuración**: La inclusión de `print("Campos disponibles:", doc.keys())` ayuda a inspeccionar la estructura de los documentos, revelando campos como `chunk` y `text_vector`, aunque el código usa `content`.



In [2]:
# Instalar dependencias (solo una vez)
!pip install python-dotenv requests openai --upgrade

# ========================
# 1. Cargar variables del entorno
# ========================
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests

load_dotenv()  # Asegúrate de haber subido el archivo .env

# Azure Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Azure OpenAI
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# ========================
# 2. Buscar documentos en Azure Search
# ========================
def search_documents(query, top_k=5):
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }
    payload = {
        "search": query,
        "top": top_k
    }
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()
    return [doc.get("content", "") for doc in results.get("value", [])]

# ========================
# 3. Llamar a Phi-4 con contexto
# ========================
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

def generate_answer(question, context):
    system_prompt = "Eres un mentor pedagógico. Responde de forma clara, breve y útil basado en el contexto."
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=500
    )
    return response.choices[0].message.content


In [3]:
pregunta = "¿Qué dijo Valentina sobre el financiamiento de la educación?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)

print("Respuesta del asistente:\n", respuesta)


Respuesta del asistente:
 Parece que no has proporcionado el contexto específico en el que Valentina habló sobre el financiamiento de la educación. Si puedes proporcionar más detalles o el texto completo donde se menciona, estaré encantado de ayudarte a resumir o responder a tu pregunta.


In [4]:
# Cambia aquí tu nueva pregunta
pregunta = "¿Qué desafíos enfrentan los estudiantes en zonas rurales según David?"

# Ejecuta las funciones con la nueva pregunta
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)

print("Respuesta del asistente:\n", respuesta)


Respuesta del asistente:
 Lamento, pero no puedo proporcionar una respuesta directa a tu pregunta ya que no has proporcionado el contexto o el texto en el que se menciona a David. Por favor, proporcióname el contexto o el texto específico donde David discute los desafíos que enfrentan los estudiantes en zonas rurales. ¡Gracias!


In [6]:
# Cambia aquí tu nueva pregunta
pregunta = "¿Qué es proyectate.info?"

# Ejecuta las funciones con la nueva pregunta
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)
respuesta = generate_answer(pregunta, contexto)

print("Respuesta del asistente:\n", respuesta)


Respuesta del asistente:
 Parece que te refieres a "projectate.info", pero no tengo información específica sobre un sitio web o recurso con ese nombre. Es posible que sea un sitio web menos conocido, un sitio personal, una página de proyecto o algo similar. Si tienes más contexto o detalles, ¡puedo intentar ayudarte mejor! Por favor, proporciona más información o aclara tu pregunta.


In [7]:
print("Documentos encontrados:\n", contexto)


Documentos encontrados:
 










In [10]:
# === 1. Instalar dependencias (si no lo hiciste) ===
# !pip install python-dotenv requests openai --upgrade

# === 2. Cargar variables de entorno ===
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import requests

load_dotenv()

# Azure Cognitive Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Azure OpenAI (Phi-4)
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# === 3. Buscar documentos en Azure Search ===
def search_documents(query, top_k=10):
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }
    payload = {
        "search": query,
        "top": top_k,
        "queryType": "simple",  # Puedes probar "semantic" si lo activas en Azure
        # "semanticConfiguration": "default",  # si activas búsqueda semántica
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()

    # Mostrar claves disponibles por documento para verificar campo correcto
    for doc in results.get("value", []):
        print("Campos disponibles:", doc.keys())
        break

    # Cambia "content" por el campo correcto si es necesario (ej. "text", "chunk", "transcription")
    return [doc.get("content", "") for doc in results.get("value", [])]

# === 4. Llamar a Phi-4 con contexto ===
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

def generate_answer(question, context):
    system_prompt = "Eres un mentor pedagógico. Responde de forma clara, breve y útil basado en el contexto."
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
        max_tokens=500
    )
    return response.choices[0].message.content

# === 5. Ejecutar ejemplo ===
pregunta = "¿Qué dijo Valentina sobre el financiamiento de la educación?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)

print("\n--- Documentos recuperados ---")
print(contexto[:1000])  # Muestra los primeros 1000 caracteres del contexto

respuesta = generate_answer(pregunta, contexto)

print("\n--- Respuesta del asistente ---")
print(respuesta)


Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])

--- Documentos recuperados ---




















--- Respuesta del asistente ---
Lo siento, pero no puedo proporcionar la información que estás buscando porque no tengo el contexto o el texto específico en el que Valentina podría haber hablado sobre el financiamiento de la educación. Si puedes proporcionar más detalles o el texto específico, estaré encantado de ayudarte a analizarlo.


In [11]:
# === 5. Ejecutar ejemplo ===
pregunta = "¿Qué debo hacer si perdí mis beneficios del credito universitario?"
documentos = search_documents(pregunta)
contexto = "\n\n".join(documentos)

print("\n--- Documentos recuperados ---")
print(contexto[:1000])  # Muestra los primeros 1000 caracteres del contexto

respuesta = generate_answer(pregunta, contexto)

print("\n--- Respuesta del asistente ---")
print(respuesta)


Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])

--- Documentos recuperados ---




















--- Respuesta del asistente ---
Si has perdido tus beneficios del crédito universitario, es importante actuar rápidamente para resolver el problema. Aquí hay algunos pasos que puedes seguir:

1. **Contacta al Centro de Ayuda Estudiantil (EAC) de tu universidad**: El EAC es el primer punto de contacto para problemas relacionados con el crédito universitario. Ellos pueden proporcionarte información específica sobre tu situación y guiarte sobre los pasos a seguir.

2. **Revisa tu información de crédito**: Verifica tu información de crédito para asegurarte de que los cambios se reflejen correctamente. Si hay errores, puedes solicitar una corrección.

3. **Revisa tu información de identidad**: Asegúrate de que tus datos de identidad sean correctos en las bases de datos del EAC. Cualquier discrepancia podría causar problemas con tu cré

In [12]:
preguntas = [
    "¿Qué es Proyectate?",
    "¿Qué dijo Valentina sobre el crédito universitario?",
    "¿Qué recomendación dio David a los estudiantes rurales?",
]

for pregunta in preguntas:
    print(f"\n🔹 Pregunta: {pregunta}")
    documentos = search_documents(pregunta)
    contexto = "\n\n".join(documentos)
    respuesta = generate_answer(pregunta, contexto)
    print("Respuesta:", respuesta)



🔹 Pregunta: ¿Qué es Proyectate?
Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])
Respuesta: Parece que la información proporcionada no es suficiente para dar una respuesta precisa. "Proyectate" podría ser una instrucción o un término en un contexto específico, como un juego, una aplicación o una conversación particular. Si puedes proporcionar más contexto o detalles, estaré encantado de ayudarte a entender mejor lo que significa "Proyectate". Por favor, proporciona más información.

🔹 Pregunta: ¿Qué dijo Valentina sobre el crédito universitario?
Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])
Respuesta: Lo siento, pero no puedo ayudarte con eso. ¿Podrías proporcionar más contexto o detalles sobre lo que dijo Valentina sobre el crédito universitario? Esto me permitirá darte una respuesta más precisa.

🔹 Pregunta: ¿Qué recomendación dio David a los estudiantes rurales?
Ca

In [14]:
preguntas = [
    "Quién es Valentina Gran de Fundación Por Una Carrera?",
    "Quién es David Leal de Innovacien?",
    "¿Qué es el chat de Inteligencia artificial que se presentó en la conversación?",
]

for pregunta in preguntas:
    print(f"\n🔹 Pregunta: {pregunta}")
    documentos = search_documents(pregunta)
    contexto = "\n\n".join(documentos)
    respuesta = generate_answer(pregunta, contexto)
    print("Respuesta:", respuesta)


🔹 Pregunta: Quién es Valentina Gran de Fundación Por Una Carrera?
Campos disponibles: dict_keys(['@search.score', 'chunk_id', 'parent_id', 'chunk', 'title', 'text_vector'])
Respuesta: Valentina Gran es una destacada defensora de la igualdad de género y la educación en Argentina. Es conocida por su trabajo en la Fundación Por Una Carrera (FPC), una organización sin fines de lucro dedicada a promover la igualdad de género en el ámbito laboral y educativo.

La Fundación Por Una Carrera, fundada por Valentina Gran en 2006, se enfoca en la creación de oportunidades para las mujeres en el mundo del trabajo y en la educación. La organización realiza diversas actividades, como la realización de seminarios, talleres y programas de capacitación, con el objetivo de empoderar a las mujeres y promover la igualdad de oportunidades.

Valentina Gran ha sido una figura clave en la promoción de políticas y programas que buscan mejorar la situación de las mujeres en Argentina y en Latinoamérica. Su trab

In [19]:
def get_query_embedding(text):
    response = client.embeddings.create(
        model=os.environ["AZURE_OPENAI_EMBEDDING"],
        input=text
    )
    return response.data[0].embedding


In [20]:
def search_documents_with_vectors(query, top_k=3):
    embedding = get_query_embedding(query)
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"

    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_SEARCH_API_KEY
    }

    payload = {
        "vectorQueries": [
            {
                "vector": embedding,
                "k": top_k,
                "fields": "content"
            }
        ],
        "select": "content,source,@search.score"
    }

    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    results = response.json()
    return [doc["content"] for doc in results.get("value", [])]


Para mejorar el código actual del cuaderno `mentoria_vocacional.ipynb`, propongo varias optimizaciones y mejoras que abordan problemas observados, mejoran la robustez, la eficiencia y la usabilidad. Las mejoras están organizadas por categoría, con explicaciones claras de los problemas identificados y las soluciones propuestas.

## 1. Manejo de Documentos Vacíos y Contexto Insuficiente

**Problema**: En la mayoría de los ejemplos, los documentos recuperados de Azure Search están vacíos (`contexto` vacío), lo que indica que el índice no contiene datos relevantes o que las consultas no están bien alineadas con el contenido indexado. Esto lleva a respuestas genéricas basadas en el conocimiento del modelo en lugar de los documentos.

**Mejoras Propuestas**:

1. **Validación de Resultados de Búsqueda**:
   - Agregar una verificación explícita para manejar casos donde no se recuperan documentos.
   - Informar al usuario cuando no se encuentran resultados y sugerir reformular la consulta.

   ```python
   def search_documents(query, top_k=10):
       ...
       results = response.json()
       documents = results.get("value", [])
       if not documents:
           print(f"No se encontraron documentos para la consulta: '{query}'")
           return []
       return [doc.get("content", "") for doc in documents]
   ```

2. **Uso de Búsqueda Semántica**:
   - Activar la búsqueda semántica (si está configurada en Azure Search) para mejorar la relevancia de los resultados, especialmente para consultas complejas.
   - Modificar el payload para incluir `queryType: "semantic"` y una configuración semántica.

   ```python
   payload = {
       "search": query,
       "top": top_k,
       "queryType": "semantic",
       "semanticConfiguration": "default"  # Asegúrate de que esté configurado en Azure
   }
   ```

3. **Depuración de Consultas**:
   - Agregar un registro de la consulta y los documentos recuperados para facilitar la depuración.
   - Mostrar un resumen del contenido recuperado (por ejemplo, los primeros 100 caracteres de cada documento).

   ```python
   def search_documents(query, top_k=10):
       ...
       documents = [doc.get("content", "") for doc in results.get("value", [])]
       print(f"Consulta: {query}")
       print(f"Documentos recuperados: {len(documents)}")
       for i, doc in enumerate(documents, 1):
           print(f"Doc {i}: {doc[:100]}...")
       return documents
   ```

**Beneficio**: Mejora la trazabilidad, permite al usuario entender por qué las respuestas no son específicas y fomenta consultas más precisas.

## 2. Implementación Completa de Búsqueda con Vectores

**Problema**: La función `search_documents_with_vectors` está definida pero no se utiliza en ningún ejemplo. Esto limita el aprovechamiento de búsquedas semánticas basadas en embeddings, que podrían mejorar la relevancia de los documentos recuperados.

**Mejoras Propuestas**:

1. **Integrar Búsqueda con Vectores**:
   - Reemplazar o complementar `search_documents` con `search_documents_with_vectors` en los ejemplos, especialmente para preguntas complejas.
   - Combinar búsqueda de texto y vectores (búsqueda híbrida) para maximizar la cobertura.

   ```python
   def search_documents_hybrid(query, top_k=10):
       embedding = get_query_embedding(query)
       url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
       headers = {
           "Content-Type": "application/json",
           "api-key": AZURE_SEARCH_API_KEY
       }
       payload = {
           "search": query,
           "top": top_k,
           "vectorQueries": [
               {
                   "vector": embedding,
                   "k": top_k,
                   "fields": "text_vector"  # Asegúrate de que el campo sea correcto
               }
           ],
           "select": "content,source,@search.score"
       }
       response = requests.post(url, headers=headers, json=payload)
       response.raise_for_status()
       results = response.json()
       documents = [doc.get("content", "") for doc in results.get("value", [])]
       if not documents:
           print(f"No se encontraron documentos para la consulta: '{query}'")
       return documents
   ```

2. **Prueba de Búsqueda con Vectores**:
   - Agregar un ejemplo que use `search_documents_with_vectors` para comparar los resultados con la búsqueda de texto simple.

   ```python
   pregunta = "¿Qué dijo Valentina sobre el financiamiento de la educación?"
   print("\n--- Búsqueda con texto ---")
   documentos_texto = search_documents(pregunta)
   print("Documentos:", documentos_texto)
   print("\n--- Búsqueda con vectores ---")
   documentos_vectores = search_documents_with_vectors(pregunta)
   print("Documentos:", documentos_vectores)
   ```

**Beneficio**: Aprovecha los embeddings para búsquedas semánticas, mejorando la relevancia de los documentos, especialmente para consultas ambiguas o con lenguaje natural.

## 3. Manejo de Errores y Robustez

**Problema**: El código no maneja bien errores potenciales, como problemas de autenticación, índices no encontrados o fallos en la API de Azure OpenAI. Esto puede causar excepciones no controladas.

**Mejoras Propuestas**:

1. **Manejo de Excepciones**:
   - Agregar bloques `try-except` en las funciones críticas (`search_documents`, `generate_answer`, `get_query_embedding`).

   ```python
   def search_documents(query, top_k=10):
       try:
           response = requests.post(url, headers=headers, json=payload)
           response.raise_for_status()
           results = response.json()
           documents = [doc.get("content", "") for doc in results.get("value", [])]
           if not documents:
               print(f"No se encontraron documentos para la consulta: '{query}'")
           return documents
       except requests.exceptions.RequestException as e:
           print(f"Error al consultar Azure Search: {e}")
           return []
       except KeyError as e:
           print(f"Error en la estructura de la respuesta: {e}")
           return []
   ```

   ```python
   def generate_answer(question, context):
       try:
           response = client.chat.completions.create(...)
           return response.choices[0].message.content
       except Exception as e:
           print(f"Error al generar respuesta con OpenAI: {e}")
           return "Lo siento, ocurrió un error al procesar la respuesta."
   ```

2. **Validación de Variables de Entorno**:
   - Verificar que todas las variables de entorno necesarias estén definidas antes de usarlas.

   ```python
   required_env_vars = [
       "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
       "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
   ]
   for var in required_env_vars:
       if not os.environ.get(var):
           raise ValueError(f"Falta la variable de entorno: {var}")
   ```

**Beneficio**: Hace que el código sea más robusto y proporciona mensajes de error claros para facilitar la depuración.

## 4. Optimización del Prompt y Respuestas

**Problema**: El `system_prompt` es genérico y no siempre produce respuestas óptimas. Además, las respuestas a veces son vagas cuando el contexto está vacío.

**Mejoras Propuestas**:

1. **Mejorar el System Prompt**:
   - Hacer el prompt más específico para el dominio de mentoría vocacional y educación.
   - Instruir al modelo para que indique explícitamente cuando no hay contexto suficiente.

   ```python
   system_prompt = """
   Eres un mentor pedagógico especializado en orientación vocacional y educación.
   Responde de forma clara, breve y útil, basándote únicamente en el contexto proporcionado.
   Si el contexto es insuficiente o no contiene información relevante, indica que no puedes responder
   con precisión y sugiere al usuario proporcionar más detalles.
   """
   ```

2. **Ajustar Parámetros del Modelo**:
   - Experimentar con `temperature` y `top_p` para equilibrar creatividad y precisión.
   - Aumentar `max_tokens` para respuestas más detalladas si es necesario.

   ```python
   response = client.chat.completions.create(
       model=AZURE_OPENAI_DEPLOYMENT_NAME,
       messages=[...],
       temperature=0.5,  # Ligeramente más creativo pero aún controlado
       max_tokens=800,   # Más espacio para respuestas detalladas
       top_p=0.9         # Controla la diversidad de las respuestas
   )
   ```

**Beneficio**: Respuestas más relevantes, concisas y alineadas con el propósito del sistema.

## 5. Documentación y Estructura del Código

**Problema**: El código carece de comentarios detallados y no está organizado de manera modular, lo que dificulta su mantenimiento y reutilización.

**Mejoras Propuestas**:

1. **Agregar Docstrings y Comentarios**:
   - Documentar cada función con docstrings que describan parámetros, retornos y propósito.
   - Agregar comentarios inline para explicar pasos complejos.

   ```python
   def search_documents(query: str, top_k: int = 10) -> list:
       """
       Busca documentos en Azure Cognitive Search basados en una consulta de texto.
       
       Args:
           query (str): Texto de la consulta de búsqueda.
           top_k (int): Número máximo de documentos a recuperar (default: 10).
       
       Returns:
           list: Lista de contenidos de los documentos recuperados.
       
       Raises:
           requests.exceptions.RequestException: Si falla la solicitud a la API.
       """
       # Construir la URL de la API
       url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
       ...
   ```

2. **Modularizar el Código**:
   - Separar el código en módulos (por ejemplo, `azure_search.py`, `openai_client.py`) para mejorar la organización.
   - Crear una función principal que orqueste la ejecución.

   ```python
   def main():
       pregunta = "¿Qué dijo Valentina sobre el financiamiento de la educación?"
       documentos = search_documents(pregunta)
       contexto = "\n\n".join(documentos)
       respuesta = generate_answer(pregunta, contexto)
       print(f"Pregunta: {pregunta}\nRespuesta: {respuesta}")

   if __name__ == "__main__":
       main()
   ```

**Beneficio**: Código más legible, mantenible y reutilizable.

## 6. Pruebas y Validación

**Problema**: No hay pruebas automatizadas ni validación de los resultados, lo que dificulta garantizar que el sistema funcione correctamente.

**Mejoras Propuestas**:

1. **Agregar Pruebas Unitarias**:
   - Usar `unittest` o `pytest` para probar las funciones clave.

   ```python
   import unittest

   class TestMentoriaVocacional(unittest.TestCase):
       def test_search_documents_empty(self):
           result = search_documents("consulta inexistente", top_k=1)
           self.assertEqual(result, [], "Debería devolver una lista vacía para consultas sin resultados")

       def test_generate_answer_no_context(self):
           result = generate_answer("Test", "")
           self.assertIn("insuficiente", result.lower(), "Debería indicar contexto insuficiente")

   if __name__ == "__main__":
       unittest.main()
   ```

2. **Validar el Índice de Azure Search**:
   - Agregar una función para verificar que el índice existe y contiene datos antes de ejecutar consultas.

   ```python
   def check_index():
       url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}?api-version=2023-07-01-Preview"
       headers = {"api-key": AZURE_SEARCH_API_KEY}
       try:
           response = requests.get(url, headers=headers)
           response.raise_for_status()
           print("Índice encontrado:", response.json()["name"])
       except requests.exceptions.RequestException as e:
           print(f"Error al verificar el índice: {e}")
           return False
       return True
   ```

**Beneficio**: Garantiza la fiabilidad del sistema y facilita la detección de errores.

## 7. Mejoras en la Experiencia del Usuario

**Problema**: Las respuestas no siempre son útiles debido a la falta de contexto, y no hay una interfaz clara para interactuar con el sistema.

**Mejoras Propuestas**:

1. **Interfaz Interactiva**:
   - Crear un bucle interactivo en el cuaderno para que los usuarios ingresen preguntas dinámicamente.

   ```python
   def interactive_mode():
       print("Sistema de Mentoría Vocacional - Escribe 'salir' para terminar")
       while True:
           pregunta = input("Ingresa tu pregunta: ")
           if pregunta.lower() == "salir":
               break
           documentos = search_documents(pregunta)
           contexto = "\n\n".join(documentos)
           respuesta = generate_answer(pregunta, contexto)
           print(f"\nRespuesta: {respuesta}\n")

   interactive_mode()
   ```

2. **Formateo de Respuestas**:
   - Usar Markdown o formato estructurado para presentar las respuestas de manera más clara.

   ```python
   from IPython.display import display, Markdown

   def display_answer(pregunta, respuesta):
       display(Markdown(f"**Pregunta**: {pregunta}\n\n**Respuesta**: {respuesta}"))
   ```

**Beneficio**: Mejora la usabilidad y hace que el sistema sea más accesible para usuarios no técnicos.

## Resumen de Mejoras

| Categoría | Mejora | Beneficio |
|-----------|--------|-----------|
| Manejo de Documentos | Validación, búsqueda semántica, depuración | Respuestas más relevantes y trazabilidad |
| Búsqueda con Vectores | Integración y búsqueda híbrida | Mejora la precisión de los resultados |
| Robustez | Manejo de errores, validación de variables | Código más estable y mensajes claros |
| Prompt y Respuestas | Prompt específico, ajustes de parámetros | Respuestas más precisas y útiles |
| Documentación | Docstrings, modularización | Código mantenible y reutilizable |
| Pruebas | Unit tests, validación de índice | Fiabilidad y detección de errores |
| Experiencia del Usuario | Interfaz interactiva, formateo | Mejor usabilidad y presentación |



In [25]:
# === 1. Instalar dependencias ===
# Nota: Descomentar la línea siguiente si necesitas instalar las dependencias
# !pip install python-dotenv requests openai --upgrade

# === 2. Importar bibliotecas y configurar entorno ===
import os
import requests
from dotenv import load_dotenv
from openai import AzureOpenAI
from IPython.display import display, Markdown
import unittest

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Validar variables de entorno requeridas
required_env_vars = [
    "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
    "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
]
for var in required_env_vars:
    if not os.environ.get(var):
        raise ValueError(f"Falta la variable de entorno: {var}")

# Configuración de Azure Cognitive Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Configuración de Azure OpenAI
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# Inicializar cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# === 3. Función de búsqueda ===
def search_documents(query: str, top_k: int = 10) -> list:
    """
    Realiza una búsqueda de texto en Azure Cognitive Search.
    Args:
        query (str): Texto de la consulta de búsqueda.
        top_k (int): Número máximo de documentos a recuperar (default: 10).
    Returns:
        list: Lista de contenidos de los documentos recuperados.
    Raises:
        requests.exceptions.RequestException: Si falla la solicitud a la API.
        KeyError: Si la respuesta tiene una estructura inesperada.
    """
    try:
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2023-07-01-Preview"
        headers = {
            "Content-Type": "application/json",
            "api-key": AZURE_SEARCH_API_KEY
        }
        payload = {
            "search": query,
            "top": top_k,
            "queryType": "semantic",
            "semanticConfiguration": "default",
            "select": "content,source,@search.score"
        }
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        results = response.json()
        documents = [doc.get("content", "") for doc in results.get("value", [])]
        # Depuración
        print(f"Consulta: {query}")
        print(f"Documentos recuperados: {len(documents)}")
        for i, doc in enumerate(documents, 1):
            print(f"Doc {i}: {doc[:100]}...")
        if not documents:
            print(f"No se encontraron documentos para la consulta: '{query}'")
        return documents
    except requests.exceptions.RequestException as e:
        print(f"Error al consultar Azure Search: {e}")
        return []
    except KeyError as e:
        print(f"Error en la estructura de la respuesta: {e}")
        return []

# === 4. Función para generar respuestas ===
def generate_answer(question: str, context: str) -> str:
    """
    Genera una respuesta usando Azure OpenAI basada en una pregunta y contexto.
    Args:
        question (str): Pregunta del usuario.
        context (str): Contexto recuperado de los documentos.
    Returns:
        str: Respuesta generada.
    Raises:
        Exception: Si falla la llamada a la API de OpenAI.
    """
    system_prompt = """
    Eres un mentor pedagógico especializado en orientación vocacional y educación.
    Responde de forma clara, breve y útil, basándote únicamente en el contexto proporcionado.
    Si el contexto es insuficiente o no contiene información relevante, indica que no puedes responder
    con precisión y sugiere al usuario proporcionar más detalles.
    """
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"
    try:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,
            max_tokens=800,
            top_p=0.9
        )
        return response.choices[0].message.content  # Corrección aquí
    except Exception as e:
        print(f"Error al generar respuesta con OpenAI: {e}")
        return "Lo siento, ocurrió un error al procesar la respuesta."

# === 5. Función para formatear respuestas ===
def display_answer(pregunta: str, respuesta: str):
    """
    Muestra la pregunta y respuesta en formato Markdown.
    Args:
        pregunta (str): Pregunta del usuario.
        respuesta (str): Respuesta generada.
    """
    display(Markdown(f"**Pregunta**: {pregunta}\n\n**Respuesta**: {respuesta}"))

# === 6. Función para verificar el índice ===
def check_index() -> bool:
    """
    Verifica si el índice de Azure Search existe y está accesible.
    Returns:
        bool: True si el índice es accesible, False en caso contrario.
    """
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}?api-version=2023-07-01-Preview"
    headers = {"api-key": AZURE_SEARCH_API_KEY}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        print("Índice encontrado:", response.json()["name"])
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error al verificar el índice: {e}")
        return False

# === 7. Modo interactivo ===
def interactive_mode():
    """
    Ejecuta un modo interactivo para que el usuario ingrese preguntas.
    """
    if not check_index():
        print("No se puede continuar debido a un problema con el índice.")
        return
    print("Sistema de Mentoría Vocacional - Escribe 'salir' para terminar")
    while True:
        pregunta = input("Ingresa tu pregunta: ")
        if pregunta.lower() == "salir":
            break
        documentos = search_documents(pregunta)
        contexto = "\n\n".join(documentos)
        respuesta = generate_answer(pregunta, contexto)
        display_answer(pregunta, respuesta)

# === 8. Pruebas unitarias ===
class TestMentoriaVocacional(unittest.TestCase):
    def test_search_documents_empty(self):
        result = search_documents("consulta inexistente", top_k=1)
        self.assertEqual(result, [], "Debería devolver una lista vacía para consultas sin resultados")

    def test_generate_answer_no_context(self):
        result = generate_answer("Test", "")
        self.assertIn("insuficiente", result.lower()) or self.assertIn("no puedes responder", result.lower(), "Debería indicar contexto insuficiente")

# === 9. Ejemplo de uso ===
def main():
    """
    Ejecuta un ejemplo de uso del sistema.
    """
    if not check_index():
        print("No se puede ejecutar el ejemplo debido a un problema con el índice.")
        return
    preguntas = [
        "¿Qué es Proyectate?",
        "¿Qué dijo Valentina sobre el crédito universitario?",
        "¿Qué recomendación dio David a los estudiantes rurales?",
        "Quién es Valentina Gran de Fundación Por Una Carrera?",
        "Quién es David Leal de Innovacien?",
        "¿Qué debo hacer si perdí mis beneficios del crédito universitario?"
    ]
    for pregunta in preguntas:
        print(f"\n🔹 Pregunta: {pregunta}")
        documentos = search_documents(pregunta)
        contexto = "\n\n".join(documentos)
        respuesta = generate_answer(pregunta, contexto)
        display_answer(pregunta, respuesta)

if __name__ == "__main__":
    # Ejecutar pruebas unitarias
    unittest.main(argv=[''], exit=False)
    # Ejecutar modo interactivo o ejemplo
    main()
    # interactive_mode() # Descomentar para usar el modo interactivo

F.
FAIL: test_generate_answer_no_context (__main__.TestMentoriaVocacional.test_generate_answer_no_context)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-2574798869.py", line 176, in test_generate_answer_no_context
    self.assertIn("insuficiente", result.lower()) or self.assertIn("no puedes responder", result.lower(), "Debería indicar contexto insuficiente")
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: 'insuficiente' not found in 'parece que te gustaría crear un contexto para un ejercicio de prueba. ¿podrías proporcionar más detalles o especificar el tema o el ámbito sobre el cual deseas crear el contexto? por ejemplo, si estás interesado en un contexto para una prueba de matemáticas, ciencias o programación, por favor, proporciona más información. ¡gracias!'

----------------------------------------------------------------------
Ran 2 tests in 3.383s

FAILED (failures=1)


Error al consultar Azure Search: 400 Client Error: Bad Request for url: https://search-vocatest.search.windows.net/indexes/rag-vocacional/docs/search?api-version=2023-07-01-Preview
Error al verificar el índice: 403 Client Error: Forbidden for url: https://search-vocatest.search.windows.net/indexes/rag-vocacional?api-version=2023-07-01-Preview
No se puede ejecutar el ejemplo debido a un problema con el índice.


In [27]:
# === 1. Instalar dependencias ===
# Nota: Descomentar la línea siguiente si necesitas instalar las dependencias
# !pip install python-dotenv requests openai --upgrade

# === 2. Importar bibliotecas y configurar entorno ===
import os
import requests
from dotenv import load_dotenv
from openai import AzureOpenAI
from IPython.display import display, Markdown
import unittest
from unittest.mock import patch, MagicMock  # Para mocks en tests

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Validar variables de entorno requeridas
required_env_vars = [
    "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
    "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
]
for var in required_env_vars:
    if not os.environ.get(var):
        raise ValueError(f"Falta la variable de entorno: {var}")

# Configuración de Azure Cognitive Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Configuración de Azure OpenAI
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# Inicializar cliente de Azure OpenAI (después de las vars para evitar cached_property issues en mocks)
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# === 3. Función de búsqueda ===
def search_documents(query: str, top_k: int = 10) -> list:
    """
    Realiza una búsqueda de texto en Azure Cognitive Search.
    """
    try:
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2024-07-01"
        headers = {
            "Content-Type": "application/json",
            "api-key": AZURE_SEARCH_API_KEY
        }
        payload = {
            "search": query,
            "top": top_k,
            "queryType": "simple",
            "select": "content,source,@search.score"
        }
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        results = response.json()
        documents = [doc.get("content", "") for doc in results.get("value", [])]
        print(f"Consulta: {query}")
        print(f"Documentos recuperados: {len(documents)}")
        for i, doc in enumerate(documents, 1):
            print(f"Doc {i}: {doc[:100]}...")
        if not documents:
            print(f"No se encontraron documentos para la consulta: '{query}'")
        return documents
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 400:
            print(f"Error 400: Configuración de búsqueda inválida. Verifica si el índice soporta el queryType. Detalles: {e.response.text}")
        else:
            print(f"Error al consultar Azure Search: {e}")
        return []
    except Exception as e:
        print(f"Error inesperado en búsqueda: {e}")
        return []

# === 4. Función para generar respuestas ===
def generate_answer(question: str, context: str) -> str:
    """
    Genera una respuesta usando Azure OpenAI basada en una pregunta y contexto.
    """
    if not context.strip():
        return "El contexto proporcionado es insuficiente para responder con precisión. Por favor, proporciona más detalles sobre tu consulta en orientación vocacional o educación."

    system_prompt = """
    Eres un mentor pedagógico especializado en orientación vocacional y educación.
    Responde de forma clara, breve y útil, basándote únicamente en el contexto proporcionado.
    Si el contexto es insuficiente o no contiene información relevante, indica que no puedes responder
    con precisión y sugiere al usuario proporcionar más detalles.
    """
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"
    try:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,
            max_tokens=800,
            top_p=0.9
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error al generar respuesta con OpenAI: {e}")
        return "Lo siento, ocurrió un error al procesar la respuesta."

# === 5. Función para formatear respuestas ===
def display_answer(pregunta: str, respuesta: str):
    """
    Muestra la pregunta y respuesta en formato Markdown.
    """
    display(Markdown(f"**Pregunta**: {pregunta}\n\n**Respuesta**: {respuesta}"))

# === 6. Función para verificar el índice ===
def check_index() -> bool:
    """
    Verifica si el índice de Azure Search existe y está accesible.
    """
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}?api-version=2024-07-01"
    headers = {"api-key": AZURE_SEARCH_API_KEY}
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 403:
            print("Error 403: API key sin permisos de lectura. Regenera la key en Azure Portal (Query Keys): https://portal.azure.com/#blade/Microsoft_Azure_Search/SearchServiceMenuBlade/Keys")
            return False
        response.raise_for_status()
        print("Índice encontrado:", response.json()["name"])
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error al verificar el índice: {e} (Status: {getattr(e.response, 'status_code', 'N/A')})")
        return False

# === 7. Modo interactivo ===
def interactive_mode():
    """
    Ejecuta un modo interactivo para que el usuario ingrese preguntas.
    """
    if not check_index():
        print("No se puede continuar debido a un problema con el índice.")
        return
    print("Sistema de Mentoría Vocacional - Escribe 'salir' para terminar")
    while True:
        pregunta = input("Ingresa tu pregunta: ")
        if pregunta.lower() == "salir":
            break
        documentos = search_documents(pregunta)
        contexto = "\n\n".join(documentos)
        respuesta = generate_answer(pregunta, contexto)
        display_answer(pregunta, respuesta)

# === 8. Pruebas unitarias ===
class TestMentoriaVocacional(unittest.TestCase):
    @patch.object(client, 'chat.completions.create')  # Mock directo en la instancia client
    def test_generate_answer_no_context(self, mock_create):
        # Dado que el if not context.strip() se activa primero, el mock no se usa, pero lo configuramos por completitud
        mock_response = MagicMock()
        mock_choice = MagicMock()
        mock_message = MagicMock()
        mock_message.content = "Mock respuesta con contexto insuficiente."
        mock_choice.message = mock_message
        mock_response.choices = [mock_choice]
        mock_create.return_value = mock_response

        result = generate_answer("Test", "")
        print(f"Resultado de la prueba: {result}")  # Debug para ver el mensaje fijo
        lower_result = result.lower()
        self.assertTrue(
            any(phrase in lower_result for phrase in ["insuficiente", "proporciona más detalles", "orientación vocacional"]),
            "Debería indicar contexto insuficiente o sugerir más detalles"
        )

    @patch('requests.post')  # Mock de requests para evitar llamadas reales
    def test_search_documents_empty(self, mock_post):
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {"value": []}
        mock_post.return_value = mock_response
        result = search_documents("consulta inexistente", top_k=1)
        self.assertEqual(result, [], "Debería devolver una lista vacía para consultas sin resultados")

# === 9. Ejemplo de uso ===
def main():
    """
    Ejecuta un ejemplo de uso del sistema.
    """
    if not check_index():
        print("No se puede ejecutar el ejemplo debido a un problema con el índice.")
        print("Solución: Verifica API key y configuración semántica en Azure Portal.")
        return
    preguntas = [
        "¿Qué es Proyectate?",
        "¿Qué dijo Valentina sobre el crédito universitario?",
        "¿Qué recomendación dio David a los estudiantes rurales?",
        "Quién es Valentina Gran de Fundación Por Una Carrera?",
        "Quién es David Leal de Innovacien?",
        "¿Qué debo hacer si perdí mis beneficios del crédito universitario?"
    ]
    for pregunta in preguntas:
        print(f"\n🔹 Pregunta: {pregunta}")
        documentos = search_documents(pregunta)
        contexto = "\n\n".join(documentos)
        respuesta = generate_answer(pregunta, contexto)
        display_answer(pregunta, respuesta)

if __name__ == "__main__":
    # Ejecutar pruebas unitarias
    unittest.main(argv=[''], exit=False, verbosity=2)
    # Ejecutar modo interactivo o ejemplo
    main()
    # interactive_mode() # Descomentar para usar el modo interactivo

test_generate_answer_no_context (__main__.TestMentoriaVocacional.test_generate_answer_no_context) ... ERROR
test_search_documents_empty (__main__.TestMentoriaVocacional.test_search_documents_empty) ... ok

ERROR: test_generate_answer_no_context (__main__.TestMentoriaVocacional.test_generate_answer_no_context)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/lib/python3.12/unittest/mock.py", line 1393, in patched
    with self.decoration_helper(patched,
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/contextlib.py", line 137, in __enter__
    return next(self.gen)
           ^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/unittest/mock.py", line 1375, in decoration_helper
    arg = exit_stack.enter_context(patching)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/contextlib.py", line 526, in enter_context
    result = _enter(cm)
             ^^^^^^^^^^
  File "/usr/lib/python3.

Consulta: consulta inexistente
Documentos recuperados: 0
No se encontraron documentos para la consulta: 'consulta inexistente'
Error 403: API key sin permisos de lectura. Regenera la key en Azure Portal (Query Keys): https://portal.azure.com/#blade/Microsoft_Azure_Search/SearchServiceMenuBlade/Keys
No se puede ejecutar el ejemplo debido a un problema con el índice.
Solución: Verifica API key y configuración semántica en Azure Portal.


In [28]:
# === 1. Instalar dependencias ===
# Nota: Descomentar la línea siguiente si necesitas instalar las dependencias
# !pip install python-dotenv requests openai --upgrade

# === 2. Importar bibliotecas y configurar entorno ===
import os
import requests
from dotenv import load_dotenv
from openai import AzureOpenAI
from IPython.display import display, Markdown
import unittest
from unittest.mock import patch, MagicMock

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Validar variables de entorno requeridas
required_env_vars = [
    "AZURE_SEARCH_API_KEY", "AZURE_SEARCH_ENDPOINT", "AZURE_SEARCH_INDEX_NAME",
    "AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_DEPLOYMENT_NAME"
]
for var in required_env_vars:
    if not os.environ.get(var):
        raise ValueError(f"Falta la variable de entorno: {var}")

# Configuración de Azure Cognitive Search
AZURE_SEARCH_API_KEY = os.environ["AZURE_SEARCH_API_KEY"]
AZURE_SEARCH_ENDPOINT = os.environ["AZURE_SEARCH_ENDPOINT"]
AZURE_SEARCH_INDEX_NAME = os.environ["AZURE_SEARCH_INDEX_NAME"]

# Configuración de Azure OpenAI
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT_NAME = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]

# Inicializar cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-08-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# === 3. Función de búsqueda ===
def search_documents(query: str, top_k: int = 10) -> list:
    """
    Realiza una búsqueda de texto en Azure Cognitive Search.
    """
    try:
        url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}/docs/search?api-version=2024-07-01"
        headers = {
            "Content-Type": "application/json",
            "api-key": AZURE_SEARCH_API_KEY
        }
        payload = {
            "search": query,
            "top": top_k,
            "queryType": "simple",
            "select": "content,source,@search.score"
        }
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        results = response.json()
        documents = [doc.get("content", "") for doc in results.get("value", [])]
        print(f"Consulta: {query}")
        print(f"Documentos recuperados: {len(documents)}")
        for i, doc in enumerate(documents, 1):
            print(f"Doc {i}: {doc[:100]}...")
        if not documents:
            print(f"No se encontraron documentos para la consulta: '{query}'")
        return documents
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 400:
            print(f"Error 400: Configuración de búsqueda inválida. Verifica si el índice soporta el queryType. Detalles: {e.response.text}")
        else:
            print(f"Error al consultar Azure Search: {e}")
        return []
    except Exception as e:
        print(f"Error inesperado en búsqueda: {e}")
        return []

# === 4. Función para generar respuestas ===
def generate_answer(question: str, context: str) -> str:
    """
    Genera una respuesta usando Azure OpenAI basada en una pregunta y contexto.
    """
    if not context.strip():
        return "El contexto proporcionado es insuficiente para responder con precisión. Por favor, proporciona más detalles sobre tu consulta en orientación vocacional o educación."

    system_prompt = """
    Eres un mentor pedagógico especializado en orientación vocacional y educación.
    Responde de forma clara, breve y útil, basándote únicamente en el contexto proporcionado.
    Si el contexto es insuficiente o no contiene información relevante, indica que no puedes responder
    con precisión y sugiere al usuario proporcionar más detalles.
    """
    user_prompt = f"Pregunta: {question}\n\nContexto:\n{context}"
    try:
        response = client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.5,
            max_tokens=800,
            top_p=0.9
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error al generar respuesta con OpenAI: {e}")
        return "Lo siento, ocurrió un error al procesar la respuesta."

# === 5. Función para formatear respuestas ===
def display_answer(pregunta: str, respuesta: str):
    """
    Muestra la pregunta y respuesta en formato Markdown.
    """
    display(Markdown(f"**Pregunta**: {pregunta}\n\n**Respuesta**: {respuesta}"))

# === 6. Función para verificar el índice ===
def check_index() -> bool:
    """
    Verifica si el índice de Azure Search existe y está accesible.
    """
    url = f"{AZURE_SEARCH_ENDPOINT}/indexes/{AZURE_SEARCH_INDEX_NAME}?api-version=2024-07-01"
    headers = {"api-key": AZURE_SEARCH_API_KEY}
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 403:
            print("Error 403: API key sin permisos de lectura. Regenera la key en Azure Portal (Query Keys): https://portal.azure.com/#blade/Microsoft_Azure_Search/SearchServiceMenuBlade/Keys")
            return False
        response.raise_for_status()
        print("Índice encontrado:", response.json()["name"])
        return True
    except requests.exceptions.RequestException as e:
        print(f"Error al verificar el índice: {e} (Status: {getattr(e.response, 'status_code', 'N/A')})")
        return False

# === 7. Modo interactivo ===
def interactive_mode():
    """
    Ejecuta un modo interactivo para que el usuario ingrese preguntas.
    """
    if not check_index():
        print("No se puede continuar debido a un problema con el índice.")
        return
    print("Sistema de Mentoría Vocacional - Escribe 'salir' para terminar")
    while True:
        pregunta = input("Ingresa tu pregunta: ")
        if pregunta.lower() == "salir":
            break
        documentos = search_documents(pregunta)
        contexto = "\n\n".join(documentos)
        respuesta = generate_answer(pregunta, contexto)
        display_answer(pregunta, respuesta)

# === 8. Pruebas unitarias ===
class TestMentoriaVocacional(unittest.TestCase):
    @patch('__main__.client.chat.completions.create')  # Mock global del path completo (fix para descriptor)
    def test_generate_answer_no_context(self, mock_create):
        # El if activa el return fijo, mock no se usa, pero configuramos por completitud
        mock_response = MagicMock()
        mock_choice = MagicMock()
        mock_message = MagicMock()
        mock_message.content = "Mock: Contexto insuficiente."
        mock_choice.message = mock_message
        mock_response.choices = [mock_choice]
        mock_create.return_value = mock_response

        result = generate_answer("Test", "")
        print(f"Resultado de la prueba (debe ser mensaje fijo): {result[:50]}...")  # Debug
        lower_result = result.lower()
        self.assertTrue(
            any(phrase in lower_result for phrase in ["insuficiente", "proporciona más detalles", "orientación vocacional"]),
            "Debería indicar contexto insuficiente o sugerir más detalles"
        )

    @patch('__main__.client.chat.completions.create')  # Test extra: con contexto (usa mock)
    def test_generate_answer_with_context(self, mock_create):
        mock_response = MagicMock()
        mock_choice = MagicMock()
        mock_message = MagicMock()
        mock_message.content = "Respuesta basada en contexto proporcionado."
        mock_choice.message = mock_message
        mock_response.choices = [mock_choice]
        mock_create.return_value = mock_response

        result = generate_answer("Test", "Contexto de ejemplo.")
        self.assertIn("basada en contexto", result.lower(), "Debería generar respuesta con contexto")

    @patch('requests.post')  # Mock de requests para evitar llamadas reales
    def test_search_documents_empty(self, mock_post):
        mock_response = MagicMock()
        mock_response.status_code = 200
        mock_response.json.return_value = {"value": []}
        mock_post.return_value = mock_response
        result = search_documents("consulta inexistente", top_k=1)
        self.assertEqual(result, [], "Debería devolver una lista vacía para consultas sin resultados")

# === 9. Ejemplo de uso ===
def main():
    """
    Ejecuta un ejemplo de uso del sistema.
    """
    if not check_index():
        print("No se puede ejecutar el ejemplo debido a un problema con el índice.")
        print("Solución: Verifica API key y configuración semántica en Azure Portal.")
        return
    preguntas = [
        "¿Qué es Proyectate?",
        "¿Qué dijo Valentina sobre el crédito universitario?",
        "¿Qué recomendación dio David a los estudiantes rurales?",
        "Quién es Valentina Gran de Fundación Por Una Carrera?",
        "Quién es David Leal de Innovacien?",
        "¿Qué debo hacer si perdí mis beneficios del crédito universitario?"
    ]
    for pregunta in preguntas:
        print(f"\n🔹 Pregunta: {pregunta}")
        documentos = search_documents(pregunta)
        contexto = "\n\n".join(documentos)
        respuesta = generate_answer(pregunta, contexto)
        display_answer(pregunta, respuesta)

if __name__ == "__main__":
    # Ejecutar pruebas unitarias
    print("Ejecutando pruebas unitarias...")
    unittest.main(argv=[''], exit=False, verbosity=2)
    print("\n" + "="*50 + "\n")
    # Ejecutar modo interactivo o ejemplo
    main()
    # interactive_mode() # Descomentar para usar el modo interactivo

test_generate_answer_no_context (__main__.TestMentoriaVocacional.test_generate_answer_no_context) ... ok
test_generate_answer_with_context (__main__.TestMentoriaVocacional.test_generate_answer_with_context) ... ok
test_search_documents_empty (__main__.TestMentoriaVocacional.test_search_documents_empty) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK


Ejecutando pruebas unitarias...
Resultado de la prueba (debe ser mensaje fijo): El contexto proporcionado es insuficiente para res...
Consulta: consulta inexistente
Documentos recuperados: 0
No se encontraron documentos para la consulta: 'consulta inexistente'


Error 403: API key sin permisos de lectura. Regenera la key en Azure Portal (Query Keys): https://portal.azure.com/#blade/Microsoft_Azure_Search/SearchServiceMenuBlade/Keys
No se puede ejecutar el ejemplo debido a un problema con el índice.
Solución: Verifica API key y configuración semántica en Azure Portal.
